# 1. Read data and do the preprocessing part (including remove stop words and punctuation )

In [2]:
### First step : loding data and turn it in to a csv form

import os
import re

import spacy
from spacy import displacy
from collections import Counter
from nltk.corpus import stopwords
#stopwords=nltk.download('stopwords')
stop = stopwords.words('english')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import MWETokenizer
import pandas as pd

train_data=[]
train_label=[]
test_data=[]
test_label=[]
data_path="./"
pos_tag = ['MD','CC','CD','DT','EX','IN','POS','PDT','PRP','RP','WDT','WP','PRP$']

def pre_process(line):
    #print "before,",line
    remove_punctuation = re.compile("[.;:!\'?,\"()\[\]]")
    remove_tag = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
    line=remove_punctuation.sub(" ",line.lower())
    line=remove_tag.sub(" ",line)
    line=[w for w in line.split() if w not in stop]
    line=" ".join(line)
    #print "after,",line
    return line 
##load train data
label=["pos","neg"]
for i in range(len(label)):
    for filename in os.listdir(data_path+"/train/"+label[i]):
        data = open(data_path+"/train/"+label[i]+"/"+filename, 'r' ).read()
        train_data.append(pre_process(data))
        train_label.append(label[i])
##load test data
for i in range(len(label)):
    for filename in os.listdir(data_path+"/test/"+label[i]):
        data = open(data_path+"/test/"+label[i]+"/"+filename, 'r' ).read()
        test_data.append(pre_process(data))
        test_label.append(label[i])
        



/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:30: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


In [14]:
import pandas as pd
a = {'review':train_data,'label':train_label}
train = pd.DataFrame(a)
b={'review':test_data,'label':test_label}
test = pd.DataFrame(b)
train.to_csv("train.csv")
test.to_csv("test.csv")


## This part is machine learning model . Use TF-IDF vectorization of word and then use Chi2 to choose top 130 features

In [16]:
from sklearn.tree import DecisionTreeClassifier 
from sklearn.feature_selection import SelectKBest  
from sklearn.feature_selection import chi2 
from sklearn import feature_extraction  
from sklearn.feature_extraction.text import TfidfVectorizer as TFIDF
from sklearn.feature_selection import mutual_info_classif 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from xgboost.sklearn import XGBClassifier

import jieba
import jieba.posseg as pseg
import jieba.analyse

import io
import re
import codecs
import numpy as np
import pandas as pd
import xgboost as xgb

def vectorize(data):    
    tfidf = TFIDF()
    fit_t = tfidf.fit_transform(data["review"])
    weight = pd.DataFrame(fit_t.toarray())
    
    word=tfidf.get_feature_names()

    return word,weight,data["label"].values

def vectorize2(data,feature):
    print data[:5]

    tfidf = TFIDF(vocabulary=feature)
    fit_t = tfidf.fit_transform(data["review"])
    weight = pd.DataFrame(fit_t.toarray())

    return weight,data["label"].values

word, x_total, y_total = vectorize(train)

f1 = SelectKBest(chi2, k=130)
f2 = f1.fit_transform(x_total, y_total)
l = f1.get_support(indices=True).tolist()
feature = {}
for i, v in enumerate(l):
    feature[word[v]] = i 
with codecs.open("selected_feature.txt","w+","utf-8") as f:
    for i in feature:
        f.write(i+"\n")
f.close()

# Use Xgboost as our model ,and got accuracy of   0.88191542 on testing data

In [17]:
ftre = set()
with codecs.open('selected_feature.txt', 'r+', 'utf-8') as f:
    for idx, line in enumerate(f.readlines()):
        line = line.strip('\n').split()
        ftre.add(line[0])
f.close()
feature = {b: a for a, b in enumerate(ftre)}

x_train, y_train = vectorize2(train,feature)
x_test, y_test = vectorize2(test, feature)

clf=XGBClassifier(
    base_score=0.5,
    colsample_bylevel=1,
    colsample_bytree=1,
    gamma=0.1,
    learning_rate=0.3,
    max_delta_step=0,
    max_depth=6,
    min_child_weight=1,
    missing=None,
    n_estimators=50,
    nthread=-1,
    objective='binary:logistic',
    reg_alpha=0.1,
    reg_lambda=0.1,
    scale_pos_weight=1,
    seed=0,
    silent=True,
    subsample=1)
clf.fit(
    x_train,
    y_train,
    eval_metric="auc",
    eval_set=[(x_train, y_train), (x_test, y_test)],
    verbose=False)

evals_result = clf.evals_result()

print "train_auc", sum(evals_result['validation_0']['auc'])/float(len(evals_result['validation_0']['auc']))
print "test_auc",sum(evals_result['validation_1']['auc'])/float(len(evals_result['validation_0']['auc']))




  label                                             review
0   pos  movie gets respect sure lot memorable quotes l...
1   pos  bizarre horror movie filled famous faces stole...
2   pos  solid unremarkable film matthau einstein wonde...
3   pos  strange feeling sit alone theater occupied par...
4   pos  probably already know 5 additional episodes ne...
  label                                             review
0   pos  based actual story john boorman shows struggle...
1   pos  gem film four production anticipated quality i...
2   pos  really like show drama romance comedy rolled o...
3   pos  best 3 experience disney themeparks certainly ...
4   pos  korean movies seen three really stuck first ex...
train_auc 0.9141816
test_auc 0.88191542


# Then I used deep learning method and still using TF-IDF vetorization  as features. The accuracy on test data is 0.839039981365

In [18]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

from tensorflow.python.keras import models
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.layers import Dropout


def get_last_layer_units_and_activation(num_classes):
    if num_classes == 2:
        activation = 'sigmoid'
        units = 1
    else:
        activation = 'softmax'
        units = num_classes
    return units, activation
def mlp_model(layers, units, dropout_rate, input_shape, num_classes):
    op_units, op_activation = get_last_layer_units_and_activation(num_classes)
    model = models.Sequential()
    model.add(Dropout(rate=dropout_rate, input_shape=input_shape))
    
    for _ in range(layers-1):
        model.add(Dense(units=units, activation='relu'))
        model.add(Dropout(rate=dropout_rate))
        
    model.add(Dense(units=op_units, activation=op_activation))
    return model

/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [52]:
def train_ngram_model(data ,learning_rate=1e-3, epochs=1000, batch_size=128, layers=2, units=64, 
                      dropout_rate=0.2):
    
    num_classes = 2
    
    # Get the data
    
    # Verify the validation labels
    '''
    unexpected_labels = [v for v in ttY if v not in range(num_classes)]
    if len(unexpected_labels):
        raise ValueError('Unexpected label values found in the validation set:'
                         ' {unexpected_labels}. Please make sure that the labels'
                         ' in the validation set are in the same range as '
                         'training labels.'.format(unexpected_labels=unexpected_labels))
    '''
    x_train, y_train, x_test,y_test=data
    
    # Create model instance
    model = mlp_model(layers, units=units, dropout_rate=dropout_rate,
                      input_shape=x_train.shape[1:], num_classes=num_classes)
    
    # Compile model with parameters
    if num_classes == 2:
        loss = 'binary_crossentropy'
    else:
        loss = 'sparse_categorical_crossentropy'
    optimizer = tf.keras.optimizers.Adam(lr=learning_rate)
    model.compile(optimizer=optimizer, loss=loss, metrics=['acc'])
    
    # Create callback for early stopping on validation loss. If the loss does
    # not decrease on two consecutive tries, stop training
    callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)]
    
    # Train and validate model
    history = model.fit(x_train, y_train, epochs=epochs, validation_data=(x_test, y_test),
                        verbose=2, batch_size=batch_size, callbacks=callbacks)
    
    # Print results
    history = history.history
    val_acc = history['val_acc'][-1]
    val_loss = history['val_loss'][-1]
    print ('Validation accuracy: {acc}, loss: {loss}'.format(
            acc=val_acc, loss=val_loss))
    
    # Save model
    model.save('IMDB_mlp_model_' + str(val_acc) + '_' + str(loss) + '.h5')
    return val_acc, val_loss

In [53]:
results = train_ngram_model((x_train, y_train, x_test,y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/1000
 - 1s - loss: 0.5327 - acc: 0.7700 - val_loss: 0.3881 - val_acc: 0.8343
Epoch 2/1000
 - 0s - loss: 0.4088 - acc: 0.8158 - val_loss: 0.3678 - val_acc: 0.8383
Epoch 3/1000
 - 0s - loss: 0.4067 - acc: 0.8161 - val_loss: 0.3658 - val_acc: 0.8386
Epoch 4/1000
 - 0s - loss: 0.4032 - acc: 0.8160 - val_loss: 0.3651 - val_acc: 0.8393
Epoch 5/1000
 - 0s - loss: 0.4003 - acc: 0.8192 - val_loss: 0.3641 - val_acc: 0.8390
Epoch 6/1000
 - 0s - loss: 0.4007 - acc: 0.8173 - val_loss: 0.3642 - val_acc: 0.8390
Epoch 7/1000
 - 0s - loss: 0.3963 - acc: 0.8200 - val_loss: 0.3647 - val_acc: 0.8390
Validation accuracy: 0.839039981365, loss: 0.36467838253


# 3.The following part is using wordvetors (downlod glove) as our feature, and using LSTM as our model , The final accurcy is 0.85  

In [1]:
TRAIN_FP = 'train.csv'
TEST_FP = 'test.csv'
MAX_NUM_WORDS = 10000

import keras
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

#set gpu config
# config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 4} ) 
# sess = tf.Session(config=config) 
# keras.backend.set_session(sess)
# print 'Using 1 GPU and 4 cores of CPU'

#read data
print 'loading data...'
train_data, test_data = pd.read_csv('train.csv'), pd.read_csv('test.csv')
texts = train_data["review"]
labels = train_data["label"]
print 'loading complete!'

/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


loading data...
loading complete!


In [2]:
print 'Tokenizing text and turing them into sequences...'
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
#sequences is a array of number[123, 456, 12 ...]
print 'process complete!'

#word index is the dictionary of sequences
word_index = tokenizer.word_index
print 'pading sequences...'
MAX_SEQUENCE_LENGTH = 200
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
print 'padding complete!'

Tokenizing text and turing them into sequences...
process complete!
pading sequences...
padding complete!


In [3]:
from keras.utils import to_categorical
#turn the labels into categotical
# label = dict()
# for i in range(len(labels)):  
#     print labels[i]
#     labels[i] = to_categorical(np.asarray(labels[i]))
mapper={"pos":1,"neg":0}
x=labels.map(mapper)
#print x
df=pd.DataFrame(data)
df['label']=x
df=df.sample(frac=1).reset_index(drop=True)
#print df[:10]
data=df.iloc[:,:200]
labels=df.iloc[:,200]
VALIDATION_SPLIT = 0.2
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

In [4]:
import os
BASE_DIR = ''
GLOVE_DIR = os.path.join(BASE_DIR, 'glove')
#read pre-trained word embedding and turn them into a dictionary call embedding_index
print 'reading pre-trained word embedding...'
embeddings_index = {}
with open('./glove.6B.300d.txt') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
print 'reading complete!' 
#
print 'making embedding matrix..'
EMBEDDING_DIM = 300
num_words = min(MAX_NUM_WORDS, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
print 'process complete!'

reading pre-trained word embedding...
reading complete!
making embedding matrix..
process complete!


In [34]:
mapper={0:1,1:0}
tmp=labels['label'].map(mapper)
#print tmp[:5],labels[:5]
labels['label2']=tmp
#print labels[:5]

0    0
1    0
2    1
3    0
4    0
Name: label, dtype: int64    label
0      1
1      1
2      0
3      1
4      1
   label  label2
0      1       0
1      1       0
2      0       1
3      1       0
4      1       0


In [35]:
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, GlobalMaxPool1D
from keras.callbacks import Callback, EarlyStopping
from sklearn.metrics import roc_auc_score
from keras.layers import Dropout
from keras.models import Model
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: {:d} - score: {:.6f}".format(epoch+1, score))


inp = Input(shape=(MAX_SEQUENCE_LENGTH, ))
x = Embedding(num_words, EMBEDDING_DIM)(inp)
x = LSTM(60, return_sequences=True,name='lstm_layer')(x)
x = GlobalMaxPool1D()(x)
x = Dropout(0.1)(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(2, activation="sigmoid")(x)

model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
mapper={0:1,1:0}
tmp=labels['label'].map(mapper)
labels['label2']=tmp

x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]

f

yt = labels[:-num_validation_samples]
yv = np.array(labels[-num_validation_samples:], dtype = float)

ra = RocAucEvaluation(validation_data=(x_val, yv), interval = 1)
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=1, verbose=1)

model.fit(x_train, yt,
          batch_size=128,
          epochs=5,
          callbacks = [ra, early_stopping],
          validation_data=(x_val, yv))

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 70s 3ms/step - loss: 0.4462 - acc: 0.7928 - val_loss: 0.2967 - val_acc: 0.8740

 ROC-AUC - epoch: 1 - score: 0.946871
Epoch 2/5
20000/20000 [==============================] - 64s 3ms/step - loss: 0.2100 - acc: 0.9222 - val_loss: 0.2946 - val_acc: 0.8758

 ROC-AUC - epoch: 2 - score: 0.947932
Epoch 3/5
20000/20000 [==============================] - 65s 3ms/step - loss: 0.1291 - acc: 0.9569 - val_loss: 0.3698 - val_acc: 0.8690

 ROC-AUC - epoch: 3 - score: 0.943647
Epoch 00003: early stopping


In [46]:
test=test_data['review']
label=pd.DataFrame(test_data['label'])
mapper={0:1,1:0}
tmp=label['label'].map(mapper)
label['label2']=tmp

In [56]:
test1 = tokenizer.texts_to_sequences(test)
test2 = pad_sequences(test1, maxlen=MAX_SEQUENCE_LENGTH)
pred = model.predict(test2)


In [ ]:
test=test_data['review']
label=pd.DataFrame(test_data['label'])
mapper={0:1,1:0}
tmp=label['label'].map(mapper)
label['label2']=tmp

In [53]:
tmp=[]
for i in range(len(pred)):
    if pred[i][0]>0.5 :
        tmp.append(1)
    else:
        tmp.append(0)
label=test_data['label']
mapper={'pos':1,'neg':0}
label=label.map(mapper)

In [55]:
from sklearn.metrics import classification_report
y_true = list(label)
y_pred = tmp
target_names = ['class 0', 'class 1']
print(classification_report(y_true, y_pred, target_names=target_names))

             precision    recall  f1-score   support

    class 0       0.88      0.81      0.84     12500
    class 1       0.82      0.89      0.86     12500

avg / total       0.85      0.85      0.85     25000

